<style>
    table {
        border-collapse: collapse; /* Para remover espaços entre as células */
        background-color: #fff; /* Define o fundo da tabela como branco */
    }
    td {
        padding: 5px; /* Adiciona espaçamento interno nas células */
    }
    td:nth-child(1) {
        width: 70%; /* Define a largura da primeira coluna */
    }
    td:nth-child(2) {
        width: 25%; /* Define a largura da segunda coluna */
    }
</style>
</head>
<body>

<table>
    <tr>
        <td>
            <span style="color:#336699; font-size: 24px;">
                <b>E</b>arth Observation Data Cubes tuned for Health Response <b>H</b>ealth <b>I</b>ndicator <b>PR</b>ocessing (EHIPR) user manual
            </span>
            <hr style="border:1px solid #0077b9;">
        </td>
        <td>
            <img src="https://raw.githubusercontent.com/Harmonize-Brazil/code-gallery/main/img/harmonize_logo.png" width=66/>
            <img src="https://raw.githubusercontent.com/Harmonize-Brazil/code-gallery/main/img/INPE_logo.jpeg" width=200/>
        </td>
    </tr>
</table>

</body>

### <span style="color:#336699" id="install"> 1. Introduction </span>
 </span>
<hr style="border:1px solid #0077b9;">


<p style='text-align: justify; font-size: 16px;'>
The <b>E</b>ODCtHRS <b>H</b>ealth <b>I</b>ndicator <b>PR</b>ocessing (ehipr) is a package developed by Harmonize Brazil Team - INPE in Python to provide a set of scripts to process health indicator data and accommodate it in the EODCtHRS component. Its flow, shown in Figure 1, begins with obtaining and reading the input data. Then, the spatial component is added to the data using the municipality grid provided by the user and it can be aggregated by spatial and temporal aggregation. By default, it uses the 2022 grid of municipalities from the Brazilian Institute of Geography and Statistics (IBGE). After spatializing the data, if the user wants to crop the data to a specific region, it can be done as defining a specific parameter when calling the spatialize_data function. Finally, it's possible to use the publish_data function to publish the data into <a href='https://geoserver.org/'>GeoServer</a> and its metadata into <a href='https://stacspec.org/en'>SpatioTemporal Asset Catalog (STAC)</a> using EODCtHRS Data PUblisher (edpu) package.

</p>

<figure style="align: center; font-size: 12px;">
  <img id="ehipr_workflow" src="https://raw.githubusercontent.com/Harmonize-Brazil/code-gallery/main/img/Health/ehipr_workflow.jpg" />
  <figcaption style='text-align: center;'><b>Figure 1</b> - ehipr workflow.</figcaption>
</figure>


### <span style="color:#336699">Example of using ehipr to process and accomodate health indicators</span>
<hr style="border:1px solid #0077b9;">

<p style='text-align: justify; font-size: 16px;'>
    Below is a guide on how to use the package:
</p>

<ol start=2>
    <li> <a href="#install" style="font-size: 16px;">Install ehipr</a>
    <li> <a href="#install" style="font-size: 16px;">Usage</a>
    <ol style='list-style-type: none;'>
        <li> <a href="#import" style="font-size: 16px;"><span style="color:#fff">3.1</span> Import ehipr</a>
        <li> <a href="#create_shp" style="font-size: 16px;"><span style="color:#fff">3.2</span> Create the shapefile delimiting the spatial aggregations</a>
        <li> <a href="#spatializing_indicator" style="font-size: 16px;"><span style="color:#fff">3.3</span> Spatializing the indicator</a>
        <li> <a href="#publishing_indicator" style="font-size: 16px;"><span style="color:#fff">3.4</span> Publishing the indicator</a>
    </ol>
    <li> <a href="#references" style="font-size: 16px;">Bibliographical references</a>
</ol>

### <span style="color:#336699" id="install"> 2. Install </span>
 </span>
<hr style="border:1px solid #0077b9;">

<p  style='text-align: justify;font-size: 16px;'>First, it's necessary to install the <a href="https://github.com/Harmonize-Brazil/ehipr">ehipr</a> package.</p>

##### <span style="color:#336699"> Note </span>
<hr style="border:1px solid #0077b9;">
<br>
<p style="font-size: 16px;">If you want to create a new Python Virtual Environment, please, follow this instruction on a terminal before installing the package:
</p>


1. First, create a new virtual environment linked to Python 3.7:

```
python3.7 -m venv venv
```

2. Activate the new environment:

```
source venv/bin/activate
```

3. Update pip, wheel and setuptools:

```
pip3 install --upgrade pip setuptools wheel
```

<br>
<hr style="border:1px solid #0077b9;">

##### <span style="color:#336699"> Installation process </span>

Use ``git`` to clone the software repository:

```
git clone https://github.com/Harmonize-Brazil/ehipr.git
```

Go to the source code folder:

```
cd ehipr
```

Install in development mode:

```
pip3 install -e .
```

<br>
<hr style="border:1px solid #0077b9;">

<p text-align='justify;' style='font-size: 16px;'>

To use the package in production mode, you need to configure a few things first, such as:
</p>

 - **Containers**

    <p text-align='justify;' style='font-size: 16px;'>
    The package was conceived with the idea of being run in an efficient and portable way, using containers. Therefore, there is a <b><span style="color:#336699;">docker compose</span></b> file in the ehipr root folder to build and run the necessary containers, which are: PostgreSQL/Postgis database, pgadmin, BDC-STAC, nginx and geoserver. Before running the command to bring up the containers, you need to edit the volumes pointed to in it to existing directories on your machine. To do this, you can use the following commands:

    - Go to the root folder of the source code:

        ```
        cd ehipr
        ```
    
    - <span style="color:red;">NOTE:</span> You must have the docker installed in your computer. <b>If you don't have</b>, you can run:
        ```
        make install_docker
         ```

    - Create the database and schema (<span style="color:red;">NOTE:</span>  before executing the following command, you should review docker-compose.yml to adjust the path of some folders to your local machine):
        ```
        make build_containers
        ```
    </p>


- **Harmonize Database**

    <p text-align='justify;' style='font-size: 16px;'>
    This database was created to store the vector layers associated with the project's health data. This database is somewhat generic and can also accommodate climate data, which is vector data. Therefore, in order to separate the data, schemas can be created. For health data, a schema called <b><span style="color:#336699;">lis</span></b> was created. If this database and schema do not exist in the environment, there is a command to resolve it.
    


    - Go to the root folder of the source code:
        ```
        cd ehipr
        ```

    - Create the database and schema:
        ```
        make harmonize_db
        ```
    </p>

<p text-align='justify;' style='font-size: 16px;'>
After these steps, the environment is already configured for the proper functioning of ehipr.
</p>

### <span style="color:#336699" id="install"> 3. Usage </span>
 </span>
<hr style="border:1px solid #0077b9;">


<p text-align='justify;' style='font-size: 16px;'>
To use the ehipr package, the user must provide a set of parameters to inform the package the characteristics of the input data. The first is the file format allowed as input data. The package can read <b>parquet</b> and <b>csv</b> files.
 
The input data must contain a set of columns associated with: spatial index, date, indicator name, spatial and temporal aggregation and indicator value. An example of the table's structure is shown in Figure 2.
</p>

<table id="spatialize_data_parameters" align="center" width="100%">
    <caption style="text-align"><b>Table 1</b> - Structure of the health data table (csv or parquet format).</caption>
    <tr style="background-color: #848484; border-radius: 1em/5em;  font-size: 16px;">
        <th style="text-align: center;" width="15%"> spatial index
        <th style="text-align: center;" width="20%"> date
        <th style="text-align: center;" width="15%"> indicator name
        <th style="text-align: center;" width="15%"> spatial aggregation
        <th style="text-align: center;" width="15%"> temporal aggregation
        <th style="text-align: center;" width="15%"> value
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> 150010 </p>
        <td> <p style="text-align: center; font-size: 16px;"> 2010-01-01 00:00:00 </p>
        <td> <p style="text-align: center; font-size: 16px;"> indi_0019 </p>
        <td> <p style="text-align: center; font-size: 16px;"> municipality </p>
        <td> <p style="text-align: center; font-size: 16px;"> year </p>
        <td> <p style="text-align: center; font-size: 16px;"> 90.67 </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> 150080 </p>
        <td> <p style="text-align: center; font-size: 16px;"> 2010-01-01 00:00:00 </p>
        <td> <p style="text-align: center; font-size: 16px;"> indi_0019 </p>
        <td> <p style="text-align: center; font-size: 16px;"> municipality </p>
        <td> <p style="text-align: center; font-size: 16px;"> year </p>
        <td> <p style="text-align: center; font-size: 16px;"> 44.9 </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> 150010 </p>
        <td> <p style="text-align: center; font-size: 16px;"> 2010-01-01 00:00:00 </p>
        <td> <p style="text-align: center; font-size: 16px;"> indi_0019 </p>
        <td> <p style="text-align: center; font-size: 16px;"> municipality </p>
        <td> <p style="text-align: center; font-size: 16px;"> month </p>
        <td> <p style="text-align: center; font-size: 16px;"> 20.00 </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> 150080 </p>
        <td> <p style="text-align: center; font-size: 16px;"> 2010-01-01 00:00:00 </p>
        <td> <p style="text-align: center; font-size: 16px;"> indi_0019 </p>
        <td> <p style="text-align: center; font-size: 16px;"> municipality </p>
        <td> <p style="text-align: center; font-size: 16px;"> month </p>
        <td> <p style="text-align: center; font-size: 16px;"> 4.00 </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> 150010 </p>
        <td> <p style="text-align: center; font-size: 16px;"> 2010-01-01 00:00:00 </p>
        <td> <p style="text-align: center; font-size: 16px;"> indi_0019 </p>
        <td> <p style="text-align: center; font-size: 16px;"> municipality </p>
        <td> <p style="text-align: center; font-size: 16px;"> week </p>
        <td> <p style="text-align: center; font-size: 16px;"> 5.00 </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> 150080 </p>
        <td> <p style="text-align: center; font-size: 16px;"> 2010-01-01 00:00:00 </p>
        <td> <p style="text-align: center; font-size: 16px;"> indi_0019 </p>
        <td> <p style="text-align: center; font-size: 16px;"> municipality </p>
        <td> <p style="text-align: center; font-size: 16px;"> week </p>
        <td> <p style="text-align: center; font-size: 16px;"> 1.00 </p>
    </tr>
</table>

<p style="font-size: 16px;">
With the input data following the pattern mentioned above, the user must provide to the package a shapefile with the grid of geometries he wants to add to the data. This file must have three columns: data spatial index, territory name and grid spatial index, which mean, respectively, the spatial index present in health data; the name of the territory associated with that spatial index; and the spatial index present in the grid.  An example of the table's structure is shown in Figure 3.
</p>

<table id="spatialize_data_parameters" align="center" width="100%">
    <caption style="text-align"><b>Table 2</b> - Structure of the shapefile with the grid of geometries.</caption>
    <tr style="background-color: #848484; border-radius: 1em/5em;  font-size: 16px;">
        <th style="text-align: center;" width="15%"> data spatial index
        <th style="text-align: center;" width="70%"> territory name
        <th style="text-align: center;" width="15%"> grid spatial index
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> 150010 </p>
        <td> <p style="text-align: center; font-size: 16px;"> Abaetetuba </p>
        <td> <p style="text-align: center; font-size: 16px;"> 1500107 </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> 150080 </p>
        <td> <p style="text-align: center; font-size: 16px;"> Ananindeua </p>
        <td> <p style="text-align: center; font-size: 16px;"> 1500800 </p>
    </tr>
</table>

### <span id="import" style="color:#336699;"> 3.1. Import Packages </span>
<hr style="border:1px solid #0077b9;">
<p style=" font-size: 16px;">
Let's load the lis and ehipr modules of ehipr package:
</p>

In [2]:
from ehipr import lis, ehipr

### <span id="create_shp" style="color:#336699">3.2. Create the shapefile delimiting the spatial aggregations</span>
<hr style="border:1px solid #0077b9;">

<p style="text-align: justify;  font-size: 16px;">
    As mentioned earlier, health data is usually provided in tabular format. To accommodate this type of data in the EODCtHRS infrastructure, it's necessary to add a geometry type to the data. To do this, the ehipr package offers a function called <b><i>create_LIS_boundaries_shp</i></b>. This function obtains the municipality grid used by the LIS team to spatially aggregate the data and match the GEOCODE<a href="" style="color:red">*</a> of the file with the current municipality grid provided by IBGE to update the name of the municipality with UTF-8 pattern. This function does not need to be executed. The script checks whether the municipal grid is available. If not, it calls the create_LIS_boundaries_shp function itself.
</p>


<p style="text-align:justify; font-size: 14px;">
    <span style="color:red;">*</span>
    The Geocode is a 7-digit code that represents Brazilian municipalities.
</p>

In [6]:
grid_path = lis.create_LIS_boundaries_shp(agg='municipality')

### <span id="spatializing_indicator" style="color:#336699">3.3. Spatializing the indicator</span>
<hr style="border:1px solid #0077b9;">

<p style="text-align: justify; font-size: 16px;">
    After updating the municipality grid, it's possible to add the geometry to the tabular data. To do this, it's necessary to inform some parameters, such as:

<table id="spatialize_data_parameters" align="center" width="100%">
    <caption style="text-align"><b>Table 3</b> - Parameter for <b>spatialize_data</b> function of ehipr module.</caption>
    <tr style="background-color: #848484; border-radius: 1em/5em;  font-size: 16px;">
        <th style="text-align: center;" width="15%"> Parameter
        <th style="text-align: center;" width="75%"> Description
        <th style="text-align: center;" width="10%"> Required
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> indicators </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a list of strings containing the identifiers of the indicators. These identifies are represented in the <b>Id</b> field in <a href="#health_indicators_table">Table 1</a>. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> input_path </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that indicates where the data is located. If the data is not available on your computer, this parameter indicates the path where the downloaded data will be stored. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> data_columns </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that indicates where the data is located. If the data is not available on your computer, this parameter indicates the path where the downloaded data will be stored. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> provider </p>
        <td> <p style="text-align: center; font-size: 16px;">  This is a string that indicates who is the data provider (ex: lis). </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> grid </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a tuple with a string that indicates where the grid is located and a dictionary with the column names. If not informed, it will use the 2022 standard grid for Brazil.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> file_crops_geom </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a list of tuples with a string indicating where the crop grid is located and a dictionary with the column names. If not informed, no crop is applied to the original grid.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> spatial_agg </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a list of strings containing the spatial aggregations you want to apply to the data. By default, its value is the spatial aggregation of the data.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> temp_agg </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a list of strings containing the temporal aggregations you want to apply to the data. By default, its value is the temporal aggregation of the data.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> github_settings </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a dictionary with the url of the github repository as the key and the token as the value. This parameter is used when the data is stored in a github repository and the user wants the package to download from there.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
</table>

<p style="text-align:justify; font-size: 16px;">
In this example, we will use the <b>Confirmed dengue cases</b> indicator provided by LIS. 
We will aggregate them by municipality and month for the regions of interest to the Harmonize project in Brazil (Northeast and North).
</p>

<p style="text-align:justify; font-size: 16px;">
First, we are going to define the parameters related to the health data, which are: 
<ul>
<li> indicators;
<li> provider;
<li> spatial_agg;
<li> temporal_agg;
<li> data_columns; and
<li> github_settings.
</p>

In [12]:
indicators = ['indi_0015']
spatial_agg = ['mun_res']
temporal_agg = ['month']
provider = 'lis'

The <b>data_columns</b> parameter is used to map the columns of the tabular data. The script recognizes and requires the following values as dictionary keys in order to read the data:

<p style="text-align:justify; font-size: 16px;">
<ul style="text-align:justify; font-size: 16px;">

<li> <b>cod</b>: it's the geocode field;
<li> <b>date</b>: it's the date field;
<li> <b>name</b>: it's the indicator name field;
<li> <b>spt_agg</b>: it's the spatial aggregation field;
<li> <b>temp_agg</b>: it's the temporal aggregation field;
<li> <b>value</b>: it's the indicator value field;

</ul>

In [9]:
data_columns = {
    'cod': 'cod', 
    'date': 'date', 
    'name': 'name',
    'spt_agg': 'agg',
    'temp_agg': 'agg_time', 
    'value': 'value'
}

<p style="text-align:justify; font-size: 16px;">
The input_path parameter indicates the location where the data will be available in parquet or csv formats. If the location indicated has no data and the github_settings parameter is entered, the input_path will be the location where the script will download and store the desired data. 
</p>

<p style="text-align:justify; font-size: 14px;">
<span style="color:red;">Note</span>: if your data is available in the specified location, you don't need to pass the github_setting parameter.</p>

In [16]:
input_path = '/home/yuri/Docker-Compose/geoserver/data/health_indicators'
github_settings = {
        'url': 'https://api.github.com/repos/rfsaldanha/bilis_harmonize/releases',
        'token': '<your_github_token>'
    }

<p style="text-align:justify; font-size: 16px;">
After defined the data parameters, we are going to define the geometry parameters. As we want the data to be clipped to the area of interest, we have to point this out to the script. To do this, we need to define a variable that receives a list with tuples indicating where the cut shapefile is stored and a dictionary pointing to the name of the columns in this file. This dictionary must contain three keys:
</p>
<ul style="text-align:justify; font-size: 16px;">
<li> <b>cod</b>: is a 6-digit string that indicates the spatial indexer used in the health data provided by LIS (IBGE geocode).
<li> <b>name</b>: it's the name of the municipalities.
<li> <b>cod_mun</b>: is a 7-digit code that represents Brazilian municipalities.
</ul>

In [8]:
shp_info = {
    'cod': 'GEOCODE',
    'name': 'NAME',
    'cod_mun': 'CD_MUN'
}

<p style="text-align:justify; font-size: 16px;">
With the dictionary mapping the names of the municipal grid columns, we can define the the tuple that points to the information about the municipal grid to be used to add the spatial components to the health data.
</p>

In [11]:
shp_grid = ('/home/yuri/teste/ehipr/ehipr/shp_malhas/default_grid/municipality/BR_municipality_2022.shp',
            shp_info)

<p style="text-align:justify; font-size: 16px;">
In addition, we can also define the dictionary mapping the names of the municipal grid columns, we can define the list of tuples used to cut the grid and obtain data only for the regions of interest.
</p>

In [18]:
crop_geoms = [
    ('../shp_malhas/northeast/northeast.shp', shp_info),
    ('../shp_malhas/north/north.shp', shp_info)
]

<p style="text-align:justify; font-size: 16px;">
Finally, we can run the <b>spatilize_data</b> function to add the geometry to the data.
</p>

In [ ]:
layers = ehipr.spatialize_data(indicators=indicators, 
                                input_path=input_path,
                                spatial_agg=spatial_agg, 
                                temp_agg=temporal_agg, 
                                file_crops_geom=crop_geoms,
                                github_settings=github_settings,
                                data_columns=data_columns,
                                provider=provider,
                                grid=shp_grid
                              )

### <span id="publishing_indicator" style="color:#336699">3.4. Publishing the indicator</span>
<hr style="border:1px solid #0077b9;">
<p style="text-align: justify; font-size: 16px;">
     Once the health data has been spatialized, it can be published on GeoServer and its metadata on STAC using the <b>edpu</b>. To do this, it's necessary to inform some parameters to the <b><i>publish_data</i></b> function, such as:

<table id="publish_data_parameters" align="center" width="100%">
    <caption style="text-align"><b>Table 4</b> - Parameter for <b>publish_data</b> function of ehipr module.</caption>
    <tr style="background-color: #848484; border-radius: 1em/5em;  font-size: 16px;">
        <th style="text-align: center;" width="15%"> Parameter
        <th style="text-align: center;" width="75%"> Description
        <th style="text-align: center;" width="10%"> Required
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> layers </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a list of dictionaries containing layer information. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> db </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that indicates the name of database the data is stored.  </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> db_schema </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the database schema name. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> gs_store </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the store name of the GeoServer service. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> time_regex </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that indicates a regex with the date and time format of the data. </p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> root_data_path </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is the root path of input_path.</p>
        <td> <p style="text-align: center; font-size: 16px;"> Yes </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> gs_service_url </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the url of the GeoServer service. By default, its value is 'http://localhost:10190/geoserver'.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> gs_username </p>
        <td> <p style="text-align: center; font-size: 16px;">  This is a string that contains the username of the GeoServer service. By default, its value is 'admin'. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> gs_password </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the password of the GeoServer service. By default, its value is 'geoserver'. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> workspace </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the workspace of the GeoServer service. By default, its value is 'bdc-lcc'. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> db_username </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the database user. By default, its value is 'postgres'.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> db_password </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the database user's password. By default, its value is 'postgres'. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> db_port </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a integer that contains the database port. By default, its value is 5432. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> stac_url </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that contains the url of the STAC service. By default, its value is 'http://localhost:8080'. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr>
        <td> <p style="text-align: center; font-size: 16px;"> hostname </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that represents the hostname IP. By default, its value is 'localhost'.</p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
    <tr style="background-color: #D8D8D8; color: #000;">
        <td> <p style="text-align: center; font-size: 16px;"> additional_path </p>
        <td> <p style="text-align: center; font-size: 16px;"> This is a string that is used to publish the data in the dev environment. By default, its value is None. </p>
        <td> <p style="text-align: center; font-size: 16px;"> No </p>
    </tr>
</table>

<p style="text-align: justify; font-size: 16px;">
As mentioned in the table <a href="#publish_data_parameters">3</a>, it's necessary to inform the parameter root_path, which is the root path of the path where the data is. 
</p>

In [24]:
root_path = '/home/yuri/Docker-Compose/geoserver/data'

Finally, you can run the <b>publish_data</b> function to publish the data as vector layers in GeoServer and its metadata in STAC.

In [ ]:
ids = ehipr.publish_data(layers=layers,
                         db='harmonize',
                         db_schema='lis',
                         gs_store='lis',
                         time_regex='regex=[0-9]{4}-[0-9]{2}-[0-9]{2}',
                         root_data_path=root_path
                        )

### <span id="references" style="color:#336699">4. Bibliographical references</span>
<hr style="border:1px solid #0077b9;">


<p id="ref_saldanha_2023" style='text-align: justify;'>Saldanha R, Xavier D, Pascoal V, Barros H, Gracie R, Magalhães M, Barcellos C (2023). bilis: An R package to calculate health indicators. <a href="https://rfsaldanha.github.io/bilis/">https://rfsaldanha.github.io/bilis/</a>, <a href="https://github.com/rfsaldanha/bilis/">https://github.com/rfsaldanha/bilis/</a>.</p>